In [1]:
%reload_ext autoreload

%autoreload 2

import math
import os

import iris
import iris.analysis.cartography
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from ml_downscaling_emulator import UKCPDatasetMetadata
from ml_downscaling_emulator.helpers import plot_over_ts
from ml_downscaling_emulator.utils import cp_model_rotated_pole, platecarree, plot_grid, open_samples_ds, show_samples, distribution_figure

In [2]:
split = "val"
dataset_name = "2.2km-coarsened-gcm-2.2km-coarsened-4x_birmingham_vorticity850_random"
gcm_derived_dataset_name = "60km-2.2km-coarsened-4x_birmingham_vorticity850_random"

run_name = "gcmx-4x_bham_vorticity850_random-fixed-gcmx-vort-grid"
checkpoint_id = 11

In [3]:
cpm_derived_samples_ds = open_samples_ds(run_name, checkpoint_id, dataset_name, split).isel(time=slice(10))
cpm_derived_samples_ds

<xarray.Dataset>
Dimensions:                     (time: 10, sample_id: 10, bnds: 2,
                                 grid_latitude: 64, grid_longitude: 64)
Coordinates:
  * time                        (time) object 1980-12-03 12:00:00 ... 1981-02...
    season                      (time) int64 dask.array<chunksize=(10,), meta=np.ndarray>
  * grid_latitude               (grid_latitude) float32 -2.49 -2.41 ... 2.55
  * grid_longitude              (grid_longitude) float32 357.9 358.0 ... 363.0
  * sample_id                   (sample_id) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  (sample_id, time) float64 dask.array<chunksize=(1, 10), meta=np.ndarray>
    time_bnds                   (sample_id, time, bnds) object dask.array<chunksize=(1, 10, 2), meta=np.ndarray>
    grid_latitude_bnds          (sample_id, time, grid_latitude, bnds) float32 dask.array<chunksize=(1, 10, 64, 2), meta=np.ndarray>
    grid_longitude_bnds         (sample_id, time, grid_longitude, bnds) float32 dask.array<chunksize=(1, 10, 64, 2), meta=np.ndarray>
    pred_pr                     (sample_id, time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(1, 10, 64, 64), meta=np.ndarray>

In [4]:
filepath = os.path.join(os.getenv("DERIVED_DATA"), 'score-sde/workdirs/subvpsde/xarray_cncsnpp_continuous', run_name, f'samples/checkpoint-{checkpoint_id}', dataset_name, split, 'predictions-1.nc')
sample1_ds = xr.open_dataset(filepath).isel(time=slice(10))
sample1_ds

<xarray.Dataset>
Dimensions:                     (time: 10, bnds: 2, grid_latitude: 64,
                                 grid_longitude: 64, sample_id: 1)
Coordinates:
  * time                        (time) object 1980-12-03 12:00:00 ... 1981-02...
    season                      (time) int64 ...
  * grid_latitude               (grid_latitude) float32 -2.49 -2.41 ... 2.55
  * grid_longitude              (grid_longitude) float32 357.9 358.0 ... 363.0
  * sample_id                   (sample_id) int64 1
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  (time) float64 ...
    time_bnds                   (time, bnds) object ...
    grid_latitude_bnds          (time, grid_latitude, bnds) float32 ...
    grid_longitude_bnds         (time, grid_longitude, bnds) float32 ...
    pred_pr                     (sample_id, time, grid_latitude, grid_longitude) float32 ...

In [5]:
filepath = os.path.join(os.getenv("DERIVED_DATA"), 'score-sde/workdirs/subvpsde/xarray_cncsnpp_continuous', run_name, f'samples/checkpoint-{checkpoint_id}', dataset_name, split, 'predictions-2.nc')
sample2_ds = xr.open_dataset(filepath).isel(time=slice(10))
sample2_ds

<xarray.Dataset>
Dimensions:                     (time: 10, bnds: 2, grid_latitude: 64,
                                 grid_longitude: 64, sample_id: 1)
Coordinates:
  * time                        (time) object 1980-12-03 12:00:00 ... 1981-02...
    season                      (time) int64 ...
  * grid_latitude               (grid_latitude) float32 -2.49 -2.41 ... 2.55
  * grid_longitude              (grid_longitude) float32 357.9 358.0 ... 363.0
  * sample_id                   (sample_id) int64 2
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  (time) float64 ...
    time_bnds                   (time, bnds) object ...
    grid_latitude_bnds          (time, grid_latitude, bnds) float32 ...
    grid_longitude_bnds         (time, grid_longitude, bnds) float32 ...
    pred_pr                     (sample_id, time, grid_latitude, grid_longitude) float32 ...

In [6]:
sample1_ds.drop("sample_id").to_netcdf("sample1.nc")
sample2_ds.drop("sample_id").to_netcdf("sample2.nc")

In [17]:
xr.concat([sample2_ds.drop_dims("sample_id"), sample1_ds.drop_dims("sample_id")], dim="sample_id")

<xarray.Dataset>
Dimensions:                     (time: 10, sample_id: 2, bnds: 2,
                                 grid_latitude: 64, grid_longitude: 64)
Coordinates:
  * time                        (time) object 1980-12-03 12:00:00 ... 1981-02...
    season                      (time) int64 0 0 0 0 0 0 0 0 0 0
  * grid_latitude               (grid_latitude) float32 -2.49 -2.41 ... 2.55
  * grid_longitude              (grid_longitude) float32 357.9 358.0 ... 363.0
Dimensions without coordinates: sample_id, bnds
Data variables:
    rotated_latitude_longitude  (sample_id, time) float64 -2.147e+09 ... -2.1...
    time_bnds                   (sample_id, time, bnds) object 1980-12-03 00:...
    grid_latitude_bnds          (sample_id, time, grid_latitude, bnds) float32 ...
    grid_longitude_bnds         (sample_id, time, grid_longitude, bnds) float32 ...

In [7]:
sample1_ds

<xarray.Dataset>
Dimensions:                     (time: 10, bnds: 2, grid_latitude: 64,
                                 grid_longitude: 64, sample_id: 1)
Coordinates:
  * time                        (time) object 1980-12-03 12:00:00 ... 1981-02...
    season                      (time) int64 0 0 0 0 0 0 0 0 0 0
  * grid_latitude               (grid_latitude) float32 -2.49 -2.41 ... 2.55
  * grid_longitude              (grid_longitude) float32 357.9 358.0 ... 363.0
  * sample_id                   (sample_id) int64 1
Dimensions without coordinates: bnds
Data variables:
    rotated_latitude_longitude  (time) float64 -2.147e+09 ... -2.147e+09
    time_bnds                   (time, bnds) object 1980-12-03 00:00:00 ... 1...
    grid_latitude_bnds          (time, grid_latitude, bnds) float32 -2.5 ... ...
    grid_longitude_bnds         (time, grid_longitude, bnds) float32 357.9 .....
    pred_pr                     (sample_id, time, grid_latitude, grid_longitude) float32 ...